<a href="https://colab.research.google.com/github/chevvakavitha/Text-Classification-with-Tensor-flow/blob/main/Text_Classification_with_Tensor_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
!pip install tensorflow_hub
!pip install tensorflow_dataset
import tensorflow_hub as hub
import tensorflow_datasets as tfds

ERROR: Could not find a version that satisfies the requirement tensorflow_dataset (from versions: none)
ERROR: No matching distribution found for tensorflow_dataset


In [ ]:
tfds.load

<function tensorflow_datasets.core.load.load(name: 'str', *, split: 'Optional[Tree[splits_lib.SplitArg]]' = None, data_dir: 'Union[None, str, os.PathLike]' = None, batch_size: 'Optional[int]' = None, shuffle_files: 'bool' = False, download: 'bool' = True, as_supervised: 'bool' = False, decoders: 'Optional[TreeDict[decode.partial_decode.DecoderArg]]' = None, read_config: 'Optional[read_config_lib.ReadConfig]' = None, with_info: 'bool' = False, builder_kwargs: 'Optional[Dict[str, Any]]' = None, download_and_prepare_kwargs: 'Optional[Dict[str, Any]]' = None, as_dataset_kwargs: 'Optional[Dict[str, Any]]' = None, try_gcs: 'bool' = False)>

In [ ]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",split=('train[:60%]', 'train[60%:]', 'test'),as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEXHWMI/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEXHWMI/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteEXHWMI/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [ ]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
!pip install tensorflow-hub

In [ ]:
 model_url = "https://tfhub.dev/google/universal-sentence-encoder/4" # Download TF Hub model from URL model = hub.load (model_url)

In [ ]:
 embedding = "https://tfhub.dev/google/universal-sentence-encoder/4"

In [ ]:
print(embedding)

https://tfhub.dev/google/universal-sentence-encoder/4


In [ ]:
hub_layer = hub.KerasLayer(embedding, input_shape = [], dtype=tf.string, trainable=True)

In [ ]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 512), dtype=float32, numpy=
array([[ 0.04070194,  0.00420416, -0.01570055, ..., -0.02733925,
         0.07720678, -0.0280134 ],
       [ 0.0230833 ,  0.03450447,  0.03191737, ..., -0.00709982,
         0.09138072,  0.00642445],
       [ 0.02236583, -0.00636588,  0.04310487, ..., -0.04315182,
         0.05988492,  0.02873515]], dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 16)                8208      
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 256806049 (979.64 MB)
Trainable params: 256806049 (979.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(100), epochs = 1, validation_data = validation_data.batch(100), verbose=1)

150/150 [==============================] - 972s 6s/step - loss: 0.4052 - accuracy: 0.7741 - val_loss: 0.2742 - val_accuracy: 0.8768


In [ ]:
results = model.evaluate(test_data.batch(100), verbose=2)
for name, values in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, values))

250/250 - 43s - loss: 0.2821 - accuracy: 0.8698 - 43s/epoch - 173ms/step
loss: 0.282
accuracy: 0.870
